In [22]:
#!/usr/bin/python
"""Set up and run simulations in Dymola, and then plot the results.
"""
__author__ = "Filip Jorissen"
__version__ = "2017-01-23"

import os
import matplotlib.pyplot as plt
import numpy as np
import re

from matplotlib import gridspec
from modelicares import gen_experiments, Experiment, SimResList, write_script, saveall

def merge_gens(gen1, gen2):
    result = []
    for i in gen1:
        result.append(i)
    for i in gen2:
        result.append(i)
    return result

def plott(sim, name, coeffTime=1, coeffVar=1, offsetVar=0, offsetTime=0, ran=[0], **args):
    var = sim[name]
    time = sim["Time"]
    if ran == [0]:
        plt.plot((time.values()[0] + offsetTime)*coeffTime,(var.values()[0]+offsetVar)*coeffVar, **args)
    else:
        plt.plot((time.values()[0][ran] + offsetTime)*coeffTime,(var.values()[0][ran]+offsetVar)*coeffVar, **args)


In [23]:
# Based on code from Kevin Davies - ModelicaRes
# Name of the Modelica script (may include the path)
FNAME = 'run_sims.mos'

# Working directory
WORKING_DIR = '/tmp/scripts'

# List of Modelica packages that should be preloaded (besides the Modelica
# Standard Library)
# Each may be a *.mo file or a path where a package.mo file resides, e.g.,
# "/opt/dymola/Modelica/Library/VehicleInterfaces 1.1.1".
#PACKAGES = ["/home/filipjorissen/dymola.mos"]

# Using dymola.mos instead of named startup script
PACKAGES = [""]   

FIGPATH="/media/psf/Home/Documents/Software/IDEAS-git/IDEAS/Resources/Images/TwinHouse/"

start = 8e6;
end = 1.23e7;

values = 5000;


# List or generator of simulations to run
EXPERIMENTS=gen_experiments(models=["IDEAS.Examples.TwinHouses.BuildingN2_Exp2"],
                             args=dict(startTime=[start],stopTime=[end], method=['\"LSodar\"'], numberOfIntervals=[values]))


# Formats in which to save the figures (e.g., ['pdf', 'eps', 'svg', 'png'])
# If the figures shouldn't be saved, specify an empty list.
FORMATS = ['pdf', 'eps']

MODELS, RESULTS_DIR = write_script(EXPERIMENTS, working_dir=WORKING_DIR,
                                       packages=PACKAGES, fname=FNAME)		
                                       
                                    






In [46]:
# For Linux:
# need to add :
# Evaluate=true;
# Advanced.EfficientMinorEvents=true;
# to run_sims.mos script for getting fast simulation!
os.system('dymola ' + FNAME)

0

In [74]:
sim["struct.Living.TAir"].times()[0][1000]

8858280.0

In [ ]:
import matplotlib.dates as md


sim = SimResList(os.path.join(RESULTS_DIR, str(1), 'dsres.mat'))
fig=plt.figure(figsize=(10,10))

ranStart=1160
ran=range(ranStart,5000)

plotStart= - md.epoch2num(1388530800)*3600*24

rangeStart=-plotStart/3600/24 + sim["struct.Living.TAir"].times()[0][ranStart]/3600/24


legendsize=10

gs = gridspec.GridSpec(4,1,height_ratios=[1,1,1,2])


ax0 = plt.subplot(gs[0])    
plt.setp(ax0.get_xticklabels(), visible=False)
plott(sim, "heaSys.heaSche.Pel_IDEAL[1]", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, coeffVar=1, label="M")
plott(sim, "heaSys.Qhea[1]", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, coeffVar=1, label="S")

plt.legend(loc=2,ncol=2,prop={'size':legendsize})
plt.ylim([0,2500])
ax0.grid(True)
plt.ylabel("Radiator thermal \n power [W]")


ax1 = plt.subplot(gs[1], sharex=ax0)    
plt.setp(ax1.get_xticklabels(), visible=False)
plott(sim, "validationDataExp2_1.living_h010cm_AT", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, coeffVar=1, label="M: h = 10 cm")
plott(sim, "validationDataExp2_1.living_h110cm_AT", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, coeffVar=1, label="M: h = 110 cm")
plott(sim, "validationDataExp2_1.living_h170cm_AT", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, coeffVar=1, label="M: h = 170 cm")
plott(sim, "struct.Living.TAir", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, coeffVar=1, label="S")
# plt.xlim([0,800/24])
plt.ylim([20,42])
plt.ylabel("Air temperature \n at height h [$^{\circ}$C]")
ax1.grid(True)
plt.legend(loc=2,ncol=2,prop={'size':legendsize})


ax2 = plt.subplot(gs[2], sharex=ax0)    
plt.setp(ax2.get_xticklabels(), visible=False)
plott(sim, "struct.W30.gainDir.y", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, coeffVar=1, label="Door dir")
plott(sim, "struct.W30.gainDif.y", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, coeffVar=1, label="Door dif")
plott(sim, "struct.W31.gainDir.y", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, coeffVar=1, label="S dir")
plott(sim, "struct.W31.gainDif.y", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, coeffVar=1, label="S dif")
plott(sim, "struct.W32.gainDir.y", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, coeffVar=1, label="W dir")
plott(sim, "struct.W32.gainDif.y", lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, coeffVar=1, label="W dif")
plt.ylabel("Exterior solar \n irradiation [W]")
# plt.xlim([0,800/24])
ax2.grid(True)
plt.legend(loc=2,ncol=3,prop={'size':legendsize})



ax3 = plt.subplot(gs[3], sharex=ax0)    
# plt.setp(ax0.get_xticklabels(), visible=False)
plott(sim, "validationDataExp2_1.living_h110cm_AT", lw=0.5, ls='--', dashes=(2, 2), color='r',  ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="M: air 110 cm")
plott(sim, "struct.heatPortCon[1].T", lw=0.5, color='r',  ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="S: air")
plott(sim, "validationDataExp2_1.living_westfacade_S_IS_T", lw=0.5, color='g', ls='--', dashes=(2, 2), ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="M: inner surface")
plott(sim, "struct.W9.layMul.port_a.T", lw=0.5, color='g',  ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="S: inner surface")
plott(sim, "validationDataExp2_1.living_westfacade_S_BL1_T", color='b', lw=0.5, ls='--', dashes=(2, 2),  ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="M: core")
plott(sim, "struct.W9.layMul.port_gain[3].T", color='b', lw=0.5, ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="S: core")
plott(sim, "validationDataExp2_1.living_westfacade_S_ES_T", lw=0.5, dashes=(2, 2), ls='--', color='m',  ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="M: outer surface")
plott(sim, "struct.W9.layMul.port_b.T", lw=0.5, color='m',  ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="S: outer surface")
plott(sim, "sim.Te", lw=0.5, ls='--', dashes=(2, 2), color='c',  ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="M: exterior")
plott(sim, "validationDataExp2_1.living_h170cm_AT", lw=0.5, ls='--', dashes=(2, 2), color='k',  ran=ran, coeffTime=1./3600/24, offsetTime=-plotStart, offsetVar=-273.15, label="M: air 170 cm")
plt.ylabel("Wall section temperatures [$^{\circ}$C]")

# plt.xlim([0,800/24])
ax3.grid(True)
plt.ylim([0,50])
plt.legend(loc=2,ncol=5,prop={'size':legendsize})
plt.xlabel("Date")
plt.xlim([rangeStart,rangeStart+33])

ax=plt.gca()
ax.xaxis_date()
xfmt = md.DateFormatter('%Y-%m-%d')
ax.xaxis.set_major_formatter(xfmt)

plt.tight_layout()

plt.savefig(FIGPATH + "TwinHouse.eps")
plt.savefig(FIGPATH + "TwinHouse.pdf")

plt.show()


In [22]:
# t0=1.45567e9


# for i in range(2):
#     fig=plt.figure(figsize=(14,4))
    
#     if i ==1:
#         deltaTime=2.5e6
#         startTime=t0
#     else:
#         deltaTime=3600*24*5
#         startTime=1.4561e9
#     endTime=startTime+deltaTime

#     times=sim["Time"].values()
#     indexStart=(np.abs(sim["Time"].values()-startTime*np.ones(len(times)))).argmin()
#     indexEnd=(np.abs(sim["Time"].values()-endTime*np.ones(len(times)))).argmin()
#     ran=range(indexStart,indexEnd)

#     times2=sim2["Time"].values()
#     indexStart2=(np.abs(sim2["Time"].values()-startTime*np.ones(len(times2)))).argmin()
#     indexEnd2=(np.abs(sim2["Time"].values()-endTime*np.ones(len(times2)))).argmin()
#     ran2=range(indexStart2,indexEnd2)


#     legendsize=10

#     gs = gridspec.GridSpec(3,1,height_ratios=[2,1,1])

#     ax0 = plt.subplot(gs[0])
#     plt.setp(ax0.get_xticklabels(), visible=False)
#     plott(sim2, "addMeas.y", ran=ran2, offsetTime=-t0, coeffTime=1./3600/24, label="Measurement")
#     plott(sim2, "addSim.y",ran=ran2, offsetTime=-t0, coeffTime=1./3600/24, label="Estimation")
#     plt.legend(loc=1,ncol=4,prop={'size':legendsize})
#     plt.ylabel('Occupancy')
#     plt.ylim([-3,25])

#     ax1 = plt.subplot(gs[1],sharex=ax0)  
#     plott(sim2, "dataBus1.FJ_vav_w04",ran=ran2, offsetTime=-t0, coeffTime=1./3600/24)
#     plott(sim2, "dataBus1.FJ_vav_w05",ran=ran2, offsetTime=-t0, coeffTime=1./3600/24)
#     plott(sim2, "dataBus1.FJ_vav_w06",ran=ran2, offsetTime=-t0, coeffTime=1./3600/24)
#     plt.setp(ax1.get_xticklabels(), visible=False)
#     plt.ylabel('VAV control \n signals y\' ')
#     plt.ylim([-0.2,1.2])
#     plt.yticks([0,1])

#     ax1 = plt.subplot(gs[2],sharex=ax0)    
#     plott(sim, "ppm_out[18]",ran=ran, offsetTime=-t0, coeffTime=1./3600/24, label="Zone")
#     plott(sim, "ppm_ext1",ran=ran, offsetTime=-t0, coeffTime=1./3600/24, label="External")
#     plt.ylabel('PPM')
#     plt.ylim([350,800])
#     plt.xlabel('Time [days]')
#     plt.xlim([(0 if i ==1 else 5),(27 if i ==1 else 10)])
#     plt.legend(loc=1,ncol=2,prop={'size':legendsize})
#     plt.yticks([400,600,800])





#     plt.tight_layout()

#     plt.savefig(os.path.join(FIGPATH, "occupancyValidation" +str(i) + ".pdf"))

#     plt.show()